In [2]:
import string, time, re, random
import numpy as np
from operator import itemgetter
from numpy.random import choice
import copy

In [7]:


class Decoder:
    POPULATION_SIZE = 100
    MAX_TRYES = 100
    MAX_LENGTH = 50
    DIRECT_PASS = 0.1
    def __init__(self, globalText,encodedText, keyLength):
        self.keyLength = keyLength
        self.encodedText = encodedText
        self.dictionary = self.create_dict(globalText)
        self.encoded_text_words = self.preprocess()
        self.current_generation = []
        self.ranks = self.set_ranks()
        self.goal = self.set_goal()
        self.best_genes = {}
        self.hist=[0]
        self.pm = 0.05
        self.pc = 0.9
        self.nm = 2
        
    def set_ranks(self):
        sum = self.POPULATION_SIZE * (self.POPULATION_SIZE + 1)/2
        ranks = [(i)/sum for i in range(self.POPULATION_SIZE)]
        ranks = sorted(ranks, reverse = True)
        return ranks
    
    def clean_data(self,text):
        text = text.upper()
        clean_text = ""
        for i in text:
            if ord('A') <= ord(i) <= ord('Z'):
                clean_text += i
            else:
                clean_text += ' '
        return clean_text.split()

    
    def create_dict(self, text):
        dictionary = set(dict.fromkeys(self.clean_data(text)))
        return dictionary

    def preprocess(self):
        words = list(self.clean_data(self.encodedText))
        return words
    
    def decrypt_text(self, key):
        decryptedText = ""
        index = 0
        for char in self.encodedText:
            if ord('a') <= ord(char) <= ord('z'):
                tempchar = char.upper()
                key_char = key[index % len(key)]
                decoded_char = chr((ord(tempchar) - ord(key_char)+26) % 26 + ord('A'))
                index += 1
                decryptedText += decoded_char.lower()
            elif ord('A') <= ord(char) <= ord('Z'):
                key_char = key[index % len(key)]
                decoded_char = chr((ord(char) - ord(key_char)+26) % 26 + ord('A'))
                index += 1
                decryptedText += decoded_char
            else:
                decryptedText += char
        return decryptedText
    
    
    def create_random_generation(self,size):
        generation = []
        size = self.POPULATION_SIZE
        for i in range(size):
            generation.append(''.join(random.sample(string.ascii_uppercase, self.keyLength)))
        bests = list(self.best_genes.keys())
        generation.extend(bests[i] for i in range(len(bests)))
        return generation
       
    def set_goal(self):
        goal = 0
        for word in self.encoded_text_words:
            goal += len(word)
        return goal
    
    def calc_fitness(self, chromosome):
        i = 0
        score = 0
        for word in self.encoded_text_words:
            decrypted_word = ""
            for char in word:
                key = chromosome[i % len(chromosome)]
                i+=1
                decrypted_word += chr((ord(char) - ord(key)+26) % 26 + ord('A'))
            if decrypted_word in self.dictionary:
                score += len(decrypted_word)
        return score
    
        
    def cross(self,par1, par2):
         
        pobability = self.pc
            
    #    if(self.MAX_LENGTH-10 < len(self.hist)):
     #       pobability = self.pc*0.7
            
      #  if(pobability>1):
       #     pobability=1
            
        if choice([0, 1], p=[1-pobability, pobability]) == 0:
            return par1, par2
        
        child1 = [' ']*self.keyLength
        child2 = [' ']*self.keyLength
       
        for i in range(0,self.keyLength):
            k = random.randint(0, 1)
            if(k):
                child1[i] = par2[i]
                child2[i] = par1[i]
            else:
                child1[i] = par1[i]
                child2[i] = par2[i]
      
        return "".join(child1), "".join(child2)
      

    def mutation(self, offspring):
       
        pobability = self.pm
        if(self.MAX_LENGTH-10 < len(self.hist)):
            pobability = self.pm*1.5
            
        if(pobability>1):
            pobability=1
        if choice([0, 1], p=[1- pobability,  pobability]) == 0:
            return offspring
        offspring = list(offspring)
        indices = sorted(random.sample(range(0,self.keyLength), self.nm))
        while(len(indices)>1):
            ind0, ind1 = random.sample(indices, 2)
            indices.remove(ind0)
            indices.remove(ind1)
            tmp = offspring[ind0]
            offspring[ind0] = offspring[ind1]
            offspring[ind1] = tmp
        return "".join(offspring)
    
    
    
    def reproduction(self,size,scores):
        new_generation = []
        while len(new_generation) < size:
            indexes = [i for i in range(0,len(scores))]
            parents_ind  = np.random.choice(indexes,2, self.ranks)
            par0 = scores[parents_ind[0]][1]
            par1 = scores[parents_ind[1]][1]
            offspring1, offspring2 = self.cross( par0, par1)
            new_gene1 = self.mutation(offspring1)
            new_gene2 = self.mutation(offspring2)
            new_generation.append(new_gene1)
            new_generation.append(new_gene2)
        return new_generation
                
    def get_best_genes(self,scores):
        num_of_direct_pass = int(self.POPULATION_SIZE * self.DIRECT_PASS)
        chosen_genes = [scores[i][1] for i in range(num_of_direct_pass)]
        return chosen_genes
    
    def update_history(self,maxVal):
        if(self.hist[len(self.hist)-1] < maxVal):
            self.hist = []
            self.hist.append(maxVal)
        else:
            self.hist.append(maxVal)
                
    def next_generation(self,scores):
        best_genes = self.get_best_genes(scores)
        size = len(self.current_generation) - len(best_genes)
        new_genes = self.reproduction(size,scores)
        self.current_generation = best_genes
        self.current_generation[len(best_genes):] = new_genes
        
         
    def evaluate_generation(self):
        fitnesses = [[self.calc_fitness(self.current_generation[i]), self.current_generation[i]] for i in range(len(self.current_generation))]
        fitnesses = sorted(fitnesses, reverse = True)
        return fitnesses
        
    def run(self):
        j=0
        self.current_generation = self.create_random_generation(self.POPULATION_SIZE - len(self.best_genes)) 
        while True:
            generation_fitness = self.evaluate_generation()
            maxVal =  generation_fitness[0][0]
            self.update_history(maxVal)
            if(len(self.hist)>self.MAX_LENGTH):
                return maxVal, generation_fitness[0][1], False
            if maxVal == self.goal:
                return maxVal,generation_fitness[0][1], True
           # if(j%10 == 0):
            #    print("i is ",j, " score is ", maxVal, "gene is ",generation_fitness[0][1])
            j+=1
            self.next_generation(generation_fitness)
           
               
    def store_best_gene(self, maxScore , best_gene ):
        if len(self.best_genes) == 0:
            self.best_genes[best_gene] = maxScore
        elif self.best_genes[max(self.best_genes, key=self.best_genes.get)] < maxScore:
            self.best_genes[best_gene] = maxScore
        if len(self.best_genes) > 1:
            del self.best_genes[min(self.best_genes, key=self.best_genes.get)]
            
    def decode(self):
        maxV = 0
        key = ""
        for i in range(self.MAX_TRYES):
            self.hist = [0]
            maxScore , best_gene , found = self.run()
            if(found):
                print("maxScore is ",maxScore , "best gene ",best_gene)
                return self.decrypt_text(best_gene)
            self.store_best_gene(maxScore , best_gene)
            if maxScore > maxV:
                maxV = maxScore
                key = best_gene
        return self.decrypt_text(key)
        
    
if __name__ == "__main__":
    
        #start = time.time()
        encodedText = open("encoded_text.txt", encoding='utf-8').read()
        globalText = open('global_text.txt', 'r', encoding='utf-8').read()
        d = Decoder(globalText,encodedText, keyLength = 14)
        decodedText = d.decode()
        print(decodedText)
        #finish = time.time()
        #print("time is ",finish-start)

maxScore is  2545 best gene  ALBERTEINSTEIN
Albert Einstein
Old Grove Rd.
Nassau Point
Peconic, Long Island

August 2nd, 1939

F.D. Roosevelt,
President of the United States,
White House
Washington, D.C.

Sir:

Some recent work by E. Fermi and L. Szilard, which has been communicated to me in manuscript, leads me to expect that the element uranium may be turned into a new and important source of energy in the immediate future. Certain aspects of the situation which has arisen seem to call for watchfulness and, if necessary, quick action on the part of the Administration. I believe therefore that it is my duty to bring to your attention the following facts and recommendations:

In the course of the last four months it has been made probable—through the work of Joliot in France as well as Fermi and Szilard in America—that it may become possible to set up a nuclear chain reaction in a large mass of uranium by which vast amounts of power and large quantities of new radium-like elements woul